In [2]:
import re
import tweepy
from tweepy import OAuthHandler
from textblob import TextBlob


# create a twitter client class


In [3]:
class TwitterClient(object):
    '''
    Generic Twitter Class for sentiment analysis.
    '''
    def __init__(self):
        '''
        Class constructor or initialization method.
        '''
        # keys and tokens from the Twitter Dev Console
        consumer_key = 'QhwdZmCoUshBbVqm1Fa1u8KzI'
        consumer_secret = '7sIToWtUk1fF2wY0xjxc591Bnkejr949LLvP4J4nWnrdr6hfjW'
        access_token = '29489350-SJOZz1RzVducii5bneoySYtch5J5mAqcKiiNue3cl'
        access_token_secret = 'r6lwFR7KtC01ukTRhkypHvW9y7fhbXPqErRxkfYUK0dAR'
  
        # attempt authentication
        try:
            # create OAuthHandler object
            self.auth = OAuthHandler(consumer_key, consumer_secret)
            # set access token and secret
            self.auth.set_access_token(access_token, access_token_secret)
            # create tweepy API object to fetch tweets
            self.api = tweepy.API(self.auth)
        except:
            print("Error: Authentication Failed")
  
    def clean_tweet(self, tweet):
        '''
        Utility function to clean tweet text by removing links, special characters
        using simple regex statements.
        '''
        return ' '.join(re.sub("(@[A-Za-z0-9]+)|([^0-9A-Za-z \t])|(\w+:\/\/\S+)", " ", tweet).split())
  
    def get_tweet_sentiment(self, tweet):
        '''
        Utility function to classify sentiment of passed tweet
        using textblob's sentiment method
        '''
        # create TextBlob object of passed tweet text
        analysis = TextBlob(self.clean_tweet(tweet))
        # set sentiment
        if analysis.sentiment.polarity > 0:
            return 'positive'
        elif analysis.sentiment.polarity == 0:
            return 'neutral'
        else:
            return 'negative'
  
    def get_tweets(self, query, count = 10):
        '''
        Main function to fetch tweets and parse them.
        '''
        # empty list to store parsed tweets
        tweets = []
  
        try:
            # call twitter api to fetch tweets
            fetched_tweets = self.api.search(q = query, count = count)
  
            # parsing tweets one by one
            for tweet in fetched_tweets:
                # empty dictionary to store required params of a tweet
                parsed_tweet = {}
  
                # saving text of tweet
                parsed_tweet['text'] = tweet.text
                # saving sentiment of tweet
                parsed_tweet['sentiment'] = self.get_tweet_sentiment(tweet.text)
  
                # appending parsed tweet to tweets list
                if tweet.retweet_count > 0:
                    # if tweet has retweets, ensure that it is appended only once
                    if parsed_tweet not in tweets:
                        tweets.append(parsed_tweet)
                else:
                    tweets.append(parsed_tweet)
  
            # return parsed tweets
            return tweets
  
        except tweepy.TweepError as e:
            # print error (if any)
            print("Error : " + str(e))


# create the main and call 

In [4]:
def main():
    # creating object of TwitterClient Class
    api = TwitterClient()
    # calling function to get tweets
    tweets = api.get_tweets(query = 'Donald Trump', count = 200)
  
    # picking positive tweets from tweets
    ptweets = [tweet for tweet in tweets if tweet['sentiment'] == 'positive']
    # percentage of positive tweets
    print("Positive tweets percentage: {} %".format(100*len(ptweets)/len(tweets)))
    # picking negative tweets from tweets
    ntweets = [tweet for tweet in tweets if tweet['sentiment'] == 'negative']
    # percentage of negative tweets
    print("Negative tweets percentage: {} %".format(100*len(ntweets)/len(tweets)))
    # percentage of neutral tweets
    print("Neutral tweets percentage: {} % \
        ".format(100*(len(tweets) -(len( ntweets )+len( ptweets)))/len(tweets)))
  
    # printing first 5 positive tweets
    print("\n\nPositive tweets:")
    for tweet in ptweets[:10]:
        print(tweet['text'])
  
    # printing first 5 negative tweets
    print("\n\nNegative tweets:")
    for tweet in ntweets[:10]:
        print(tweet['text'])
  
if __name__ == "__main__":
    # calling main function
    main()

Positive tweets percentage: 44.067796610169495 %
Negative tweets percentage: 20.338983050847457 %
Neutral tweets percentage: 35.59322033898305 %         


Positive tweets:
RT @JackPosobiec: If Donald Trump was caught funding chimeric superviruses in a Russian bioweapons lab it would be the top story in the wor…
RT @nytimes: Speaker Nancy Pelosi moved on Wednesday to bar two of Donald Trump’s most vociferous Republican defenders in Congress from joi…
RT @KariLake: So excited and honored to speak at the Rally to Save our Elections with President Donald J Trump this Saturday in downtown Ph…
RT @Mhambi_M: Donald Trump nominated over 230 individuals to the federal judiciary during his presidency.

Senate Republicans confirmed at…
RT @NoahBookbinder: $2.4 billion is a staggering amount of money for Donald Trump's company to have made during his presidency--with much o…
RT @RevChasC: @CappaReena I thought he did very, very well! I was in tears as I thanked God for deliverance from that demon